# Data Engineering 1 Lab 3 

## GitHub Link

https://github.com/eanelson01/Lab3

## Question 2

### 2a.

In [1]:
import numpy as np
import pandas as pd
import requests
import os
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
import psycopg2
import zipfile
import io
from sqlalchemy import create_engine

### 2b.

url = 'https://databank.worldbank.org/data/download/ESG_CSV.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

url = 'https://v-dem.net/media/datasets/V-Dem-CY-Core_csv_v13.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

Load the 'V-Dem-CY-Corev13.csv' file as vdem , the 'ESGData.csv' file as wb , and the 'ESGCountry.csv' files as
country .

In [2]:
vdem = pd.read_csv('V-Dem-CY-Core-v13.csv')
wb = pd.read_csv('ESGData.csv')
country = pd.read_csv('ESGCountry.csv')

### 2c.

#### VDEM

In [3]:
# Keep only the 'country_text_id', 'country_name','year', 'v2x_polyarchy' columns.
vdem = vdem[['country_text_id', 'country_name', 'year', 'v2x_polyarchy']]

# Use the .query() method to keep only the rows in which year is greater than or equal to 1960 and less than or equal to 2021.

vdem = vdem.query("year >= 1960 & year <= 2021")

In [4]:
# Rename 'country_text_id' to 'country_code', 'country_name' to 'country_name_vdem', and 'v2x_polyarchy' to 'democracy'.

vdem.rename({'country_text_id': 'country_code', 
             'country_name': 'country_name_vdem',
             'v2x_polyarchy': 'democracy'},
            axis = 1,
           inplace = True)
vdem

,country_code,country_name_vdem,year,democracy
171,MEX,Mexico,1960,0.232
172,MEX,Mexico,1961,0.234
173,MEX,Mexico,1962,0.233
174,MEX,Mexico,1963,0.233
175,MEX,Mexico,1964,0.231
...,...,...,...,...
26150,ZZB,Zanzibar,2017,0.267
26151,ZZB,Zanzibar,2018,0.268
26152,ZZB,Zanzibar,2019,0.266
26153,ZZB,Zanzibar,2020,0.258


In [5]:
# Sort the rows by 'country_code' and 'year' in ascending order.
vdem.sort_values(by = ['country_code', 'year'],
                 inplace = True)
vdem

,country_code,country_name_vdem,year,democracy
5433,AFG,Afghanistan,1960,0.080
5434,AFG,Afghanistan,1961,0.083
5435,AFG,Afghanistan,1962,0.082
5436,AFG,Afghanistan,1963,0.085
5437,AFG,Afghanistan,1964,0.137
...,...,...,...,...
26150,ZZB,Zanzibar,2017,0.267
26151,ZZB,Zanzibar,2018,0.268
26152,ZZB,Zanzibar,2019,0.266
26153,ZZB,Zanzibar,2020,0.258


### 2d. country

In [6]:
# Keep only the 'Country Code', 'Table Name', 'Long Name', 'Currency Unit', 'Region', and 'Income Group' columns.

country = country[['Country Code', 'Table Name', 'Long Name', 'Currency Unit', 'Region', 'Income Group']]

In [7]:
# Rename 'Country Code' to 'country_code', 'Table Name' to 'country_name_wb',
# 'Long Name' to 'country_longname', 'Currency Unit' to 'currency_unit', 'Region' to
# 'region', and 'Income Group' to 'income_group'.

country.rename({'Country Code': 'country_code',
               'Table Name': 'country_name_wb',
               'Long Name': 'country_longname',
               'Currency Unit': 'currency_unit',
               'Region': 'region',
               'Income Group': 'income_group'},
                axis = 1,
                inplace = True)
country

,country_code,country_name_wb,country_longname,currency_unit,region,income_group
0,AFG,Afghanistan,Islamic State of Afghanistan,Afghan afghani,South Asia,Low income
1,AGO,Angola,People's Republic of Angola,Angolan kwanza,Sub-Saharan Africa,Lower middle income
2,ALB,Albania,Republic of Albania,Albanian lek,Europe & Central Asia,Upper middle income
3,AND,Andorra,Principality of Andorra,Euro,Europe & Central Asia,High income
4,ARB,Arab World,Arab World,NaN,NaN,NaN
...,...,...,...,...,...,...
234,WSM,Samoa,Samoa,Samoan tala,East Asia & Pacific,Lower middle income
235,YEM,"Yemen, Rep.",Republic of Yemen,Yemeni rial,Middle East & North Africa,Low income
236,ZAF,South Africa,Republic of South Africa,South African rand,Sub-Saharan Africa,Upper middle income
237,ZMB,Zambia,Republic of Zambia,New Zambian kwacha,Sub-Saharan Africa,Lower middle income


In [8]:
noncountries = ["Arab World", "Central Europe and the Baltics",
                 "Caribbean small states",
                 "East Asia & Pacific (excluding high income)",
                 "Early-demographic dividend","East Asia & Pacific",
                 "Europe & Central Asia (excluding high income)",
                 "Europe & Central Asia", "Euro area",
                 "European Union","Fragile and conflict affected situations",
                 "High income",
                 "Heavily indebted poor countries (HIPC)","IBRD only",
                 "IDA & IBRD total",
                 "IDA total","IDA blend","IDA only",
                 "Latin America & Caribbean (excluding high income)",
                 "Latin America & Caribbean",
                 "Least developed countries: UN classification",
                 "Low income","Lower middle income","Low & middle income",
                 "Late-demographic dividend","Middle East & North Africa",
                 "Middle income",
                 "Middle East & North Africa (excluding high income)",
                 "North America","OECD members",
                 "Other small states","Pre-demographic dividend",
                 "Pacific island small states",
                 "Post-demographic dividend",
                 "Sub-Saharan Africa (excluding high income)",
                 "Sub-Saharan Africa",
                 "Small states","East Asia & Pacific (IDA & IBRD)",
                 "Europe & Central Asia (IDA & IBRD)",
                 "Latin America & Caribbean (IDA & IBRD)",
                 "Middle East & North Africa (IDA & IBRD)","South Asia",
                 "South Asia (IDA & IBRD)",
                 "Sub-Saharan Africa (IDA & IBRD)",
                 "Upper middle income", "World"]
country = country.query("country_name_wb not in @noncountries")
country

,country_code,country_name_wb,country_longname,currency_unit,region,income_group
0,AFG,Afghanistan,Islamic State of Afghanistan,Afghan afghani,South Asia,Low income
1,AGO,Angola,People's Republic of Angola,Angolan kwanza,Sub-Saharan Africa,Lower middle income
2,ALB,Albania,Republic of Albania,Albanian lek,Europe & Central Asia,Upper middle income
3,AND,Andorra,Principality of Andorra,Euro,Europe & Central Asia,High income
5,ARE,United Arab Emirates,United Arab Emirates,U.A.E. dirham,Middle East & North Africa,High income
...,...,...,...,...,...,...
234,WSM,Samoa,Samoa,Samoan tala,East Asia & Pacific,Lower middle income
235,YEM,"Yemen, Rep.",Republic of Yemen,Yemeni rial,Middle East & North Africa,Low income
236,ZAF,South Africa,Republic of South Africa,South African rand,Sub-Saharan Africa,Upper middle income
237,ZMB,Zambia,Republic of Zambia,New Zambian kwacha,Sub-Saharan Africa,Lower middle income


### 2e. wb

In [9]:
# Keep only the columns named 'Country Code', 'Country Name', and 'Indicator Code',
# or begin with '19' or '20'. (Don't type in all the years individually. Instead, use code
# that finds all columns that begin '19' or '20'.)

all_cols = wb.columns
selected_cols = ['Country Code', 'Country Name', 'Indicator Code'] + [x for x in all_cols if (x[:2] == '19' or x[:2] == '20')]

wb = wb[selected_cols]
wb

,Country Code,Country Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,ARB,Arab World,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,86.406487,86.705717,86.942778,87.228705,87.390856,87.617862,87.798740,87.948264,88.092536,NaN
1,ARB,Arab World,EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,89.340705,88.832276,89.053852,89.539016,90.662754,89.176939,90.352802,90.635050,90.845661,NaN
2,ARB,Arab World,NY.ADJ.DRES.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.598544,10.263684,6.217421,5.190735,6.458100,8.451699,7.351742,4.347213,NaN,NaN
3,ARB,Arab World,NY.ADJ.DFOR.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.062081,0.084113,0.098458,0.093562,0.095137,0.050799,0.061584,0.077949,NaN,NaN
4,ARB,Arab World,AG.LND.AGRI.ZS,NaN,30.981414,30.982663,31.007054,31.018001,31.042466,31.0504,...,39.838650,39.834421,39.872575,39.937814,39.984452,39.969738,39.907031,39.973290,39.970742,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16964,ZWE,Zimbabwe,ER.PTD.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27.214542,27.214585,27.214585,27.214747,27.214747,27.214747,27.214747
16965,ZWE,Zimbabwe,AG.LND.FRLS.HA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16966,ZWE,Zimbabwe,SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.994000,4.770000,5.282000,5.796000,6.279000,6.784000,7.370000,7.898000,8.067000,7.945000
16967,ZWE,Zimbabwe,SP.UWT.TFRT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.382129,10.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Rename 'Country Code' to'country_code', 'Country Name' to 'country_name_wb',
# and 'Indicator Code' to 'feature'.

wb.rename({'Country Code': 'country_code',
          'Country Name': 'country_name_wb',
          'Indicator Code': 'feature'},
          axis = 1,
         inplace = True)
wb.head()

,country_code,country_name_wb,feature,1960,1961,1962,1963,1964,1965,1966,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,ARB,Arab World,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,86.406487,86.705717,86.942778,87.228705,87.390856,87.617862,87.798740,87.948264,88.092536,NaN
1,ARB,Arab World,EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,89.340705,88.832276,89.053852,89.539016,90.662754,89.176939,90.352802,90.635050,90.845661,NaN
2,ARB,Arab World,NY.ADJ.DRES.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.598544,10.263684,6.217421,5.190735,6.458100,8.451699,7.351742,4.347213,NaN,NaN
3,ARB,Arab World,NY.ADJ.DFOR.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.062081,0.084113,0.098458,0.093562,0.095137,0.050799,0.061584,0.077949,NaN,NaN
4,ARB,Arab World,AG.LND.AGRI.ZS,NaN,30.981414,30.982663,31.007054,31.018001,31.042466,31.0504,...,39.838650,39.834421,39.872575,39.937814,39.984452,39.969738,39.907031,39.973290,39.970742,NaN


In [11]:
# Type noncountries.remove('World') to remove 'World' from the
# noncountries list. (We want to keep the total world data for now). Then use the
# .query() method to remove the rows in which 'country_name_wb' is equal to one
# of the other entries in the noncountries list.

noncountries.remove('World')

wb = wb.query("country_name_wb not in @noncountries")
wb

,country_code,country_name_wb,feature,1960,1961,1962,1963,1964,1965,1966,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
3195,WLD,World,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,61.095607,62.372600,63.662869,65.005668,66.321819,67.696517,68.921601,70.184805,71.331487,NaN
3196,WLD,World,EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,85.817366,86.306969,87.025241,88.192385,89.019548,89.893733,90.194716,90.483629,91.413941,NaN
3197,WLD,World,NY.ADJ.DRES.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.778802,1.520162,0.820341,0.717655,0.919547,1.246028,1.087624,0.790953,NaN,NaN
3198,WLD,World,NY.ADJ.DFOR.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.061422,0.069023,0.071087,0.073026,0.069027,0.048838,0.049834,0.058687,NaN,NaN
3199,WLD,World,AG.LND.AGRI.ZS,NaN,35.879317,35.95247,36.035383,36.117043,36.213941,36.294321,...,36.790232,36.789051,36.620496,36.587014,36.834001,36.738458,36.762648,36.730920,36.841665,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16964,ZWE,Zimbabwe,ER.PTD.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27.214542,27.214585,27.214585,27.214747,27.214747,27.214747,27.214747
16965,ZWE,Zimbabwe,AG.LND.FRLS.HA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16966,ZWE,Zimbabwe,SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.994000,4.770000,5.282000,5.796000,6.279000,6.784000,7.370000,7.898000,8.067000,7.945000
16967,ZWE,Zimbabwe,SP.UWT.TFRT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.382129,10.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
replace_map = {
 "AG.LND.AGRI.ZS": "agricultural_land",
 "AG.LND.FRST.ZS": "forest_area",
 "AG.PRD.FOOD.XD": "food_production_index",
 "CC.EST": "control_of_corruption",
 "EG.CFT.ACCS.ZS": "access_to_clean_fuels_and_technologies_for_cooking",
 "EG.EGY.PRIM.PP.KD": "energy_intensity_level_of_primary_energy",
 "EG.ELC.ACCS.ZS": "access_to_electricity",
 "EG.ELC.COAL.ZS": "electricity_production_from_coal_sources",
 "EG.ELC.RNEW.ZS": "renewable_electricity_output",
 "EG.FEC.RNEW.ZS": "renewable_energy_consumption",
 "EG.IMP.CONS.ZS": "energy_imports",
 "EG.USE.COMM.FO.ZS": "fossil_fuel_energy_consumption",
 "EG.USE.PCAP.KG.OE": "energy_use",
 "EN.ATM.CO2E.PC": "co2_emissions",
 "EN.ATM.METH.PC": "methane_emissions",
 "EN.ATM.NOXE.PC": "nitrous_oxide_emissions",
 "EN.ATM.PM25.MC.M3": "pm2_5_air_pollution",
 "EN.CLC.CDDY.XD": "cooling_degree_days",
 "EN.CLC.GHGR.MT.CE": "ghg_net_emissions",
 "EN.CLC.HEAT.XD": "heat_index_35",
 "EN.CLC.MDAT.ZS": "droughts",
 "EN.CLC.PRCP.XD": "maximum_5-day_rainfall",
 "EN.CLC.SPEI.XD": "mean_drought_index",
"EN.MAM.THRD.NO": "mammal_species",
 "EN.POP.DNST": "population_density",
 "ER.H2O.FWTL.ZS": "annual_freshwater_withdrawals",
 "ER.PTD.TOTL.ZS": "terrestrial_and_marine_protected_areas",
 "GB.XPD.RSDV.GD.ZS": "research_and_development_expenditure",
 "GE.EST": "government_effectiveness",
 "IC.BUS.EASE.XQ": "ease_of_doing_business_rank",
 "IC.LGL.CRED.XQ": "strength_of_legal_rights_index",
 "IP.JRN.ARTC.SC": "scientific_and_technical_journal_articles",
 "IP.PAT.RESD": "patent_applications",
 "IT.NET.USER.ZS": "individuals_using_the_internet",
 "NV.AGR.TOTL.ZS": "agriculture",
 "NY.ADJ.DFOR.GN.ZS": "net_forest_depletion",
 "NY.ADJ.DRES.GN.ZS": "natural_resources_depletion",
 "NY.GDP.MKTP.KD.ZG": "gdp_growth",
 "PV.EST": "political_stability_and_absence_of_violence",
 "RL.EST": "rule_of_law",
 "RQ.EST": "regulatory_quality",
 "SE.ADT.LITR.ZS": "literacy_rate",
 "SE.ENR.PRSC.FM.ZS": "gross_school_enrollment",
 "SE.PRM.ENRR": "primary_school_enrollment",
 "SE.XPD.TOTL.GB.ZS": "government_expenditure_on_education",
 "SG.GEN.PARL.ZS": "proportion_of_seats_held_by_women_in_national_parliament",
 "SH.DTH.COMM.ZS": "cause_of_death",
 "SH.DYN.MORT": "mortality_rate",
 "SH.H2O.SMDW.ZS": "people_using_safely_managed_drinking_water_services",
 "SH.MED.BEDS.ZS": "hospital_beds",
 "SH.STA.OWAD.ZS": "prevalence_of_overweight",
 "SH.STA.SMSS.ZS": "people_using_safely_managed_sanitation_services",
 "SI.DST.FRST.20": "income_share_held_by_lowest_20pct",
 "SI.POV.GINI": "gini_index",
 "SI.POV.NAHC": "poverty_headcount_ratio_at_national_poverty_lines",
 "SI.SPR.PCAP.ZG": "annualized_average_growth_rate_in_per_capita_real_survey",
 "SL.TLF.0714.ZS": "children_in_employment",
 "SL.TLF.ACTI.ZS": "labor_force_participation_rate",
 "SL.TLF.CACT.FM.ZS": "ratio_of_female_to_male_labor_force_participation_rate",
 "SL.UEM.TOTL.ZS": "unemployment",
 "SM.POP.NETM": "net_migration",
 "SN.ITK.DEFC.ZS": "prevalence_of_undernourishment",
 "SP.DYN.LE00.IN": "life_expectancy_at_birth",
 "SP.DYN.TFRT.IN": "fertility_rate",
 "SP.POP.65UP.TO.ZS": "population_ages_65_and_above",
 "SP.UWT.TFRT": "unmet_need_for_contraception",
 "VA.EST": "voice_and_accountability",
 "EN.CLC.CSTP.ZS": "coastal_protection",
 "SD.ESR.PERF.XQ": "economic_and_social_rights_performance_score",
 "EN.CLC.HDDY.XD": "heating_degree_days",
 "EN.LND.LTMP.DC": "land_surface_temperature",
 "ER.H2O.FWST.ZS": "freshwater_withdrawal",
 "EN.H2O.BDYS.ZS": "water_quality",
 "AG.LND.FRLS.HA": "tree_cover_loss",
}

In [13]:
# The features in this dataset are given strange and incomprehensible codes such as
# 'EG.CFT.ACCS.ZS'. Use the replace_map dictionary, defined below, to recode all
# of these values with more descriptive names for each feature.

wb.replace({'feature': replace_map}, inplace = True)

/tmp/ipykernel_316/3850477962.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wb.replace({'feature': replace_map}, inplace = True)


In [14]:
wb

,country_code,country_name_wb,feature,1960,1961,1962,1963,1964,1965,1966,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
3195,WLD,World,access_to_clean_fuels_and_technologies_for_coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,61.095607,62.372600,63.662869,65.005668,66.321819,67.696517,68.921601,70.184805,71.331487,NaN
3196,WLD,World,access_to_electricity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,85.817366,86.306969,87.025241,88.192385,89.019548,89.893733,90.194716,90.483629,91.413941,NaN
3197,WLD,World,natural_resources_depletion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.778802,1.520162,0.820341,0.717655,0.919547,1.246028,1.087624,0.790953,NaN,NaN
3198,WLD,World,net_forest_depletion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.061422,0.069023,0.071087,0.073026,0.069027,0.048838,0.049834,0.058687,NaN,NaN
3199,WLD,World,agricultural_land,NaN,35.879317,35.95247,36.035383,36.117043,36.213941,36.294321,...,36.790232,36.789051,36.620496,36.587014,36.834001,36.738458,36.762648,36.730920,36.841665,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16964,ZWE,Zimbabwe,terrestrial_and_marine_protected_areas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27.214542,27.214585,27.214585,27.214747,27.214747,27.214747,27.214747
16965,ZWE,Zimbabwe,tree_cover_loss,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16966,ZWE,Zimbabwe,unemployment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.994000,4.770000,5.282000,5.796000,6.279000,6.784000,7.370000,7.898000,8.067000,7.945000
16967,ZWE,Zimbabwe,unmet_need_for_contraception,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.382129,10.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 2f.

In [15]:
# melting the dataframe
wb = pd.melt(wb, ['country_code', 'country_name_wb', 'feature'], [str(x) for x in all_cols if (x[:2] == '19' or x[:2] == '20')])

In [16]:
wb.rename({'variable': 'year'}, 
         axis = 1,
         inplace = True)
wb.head()

,country_code,country_name_wb,feature,year,value
0,WLD,World,access_to_clean_fuels_and_technologies_for_coo...,1960,NaN
1,WLD,World,access_to_electricity,1960,NaN
2,WLD,World,natural_resources_depletion,1960,NaN
3,WLD,World,net_forest_depletion,1960,NaN
4,WLD,World,agricultural_land,1960,NaN


In [17]:
wb = wb.pivot_table(index = ['country_code', 'country_name_wb', 'year'], 
                columns = 'feature',
                values = 'value')
wb

feature                            access_to_clean_fuels_and_technologies_for_cooking  \
country_code country_name_wb year                                                       
AFG          Afghanistan     1960                                                NaN    
                             1961                                                NaN    
                             1962                                                NaN    
                             1963                                                NaN    
                             1964                                                NaN    
...                                                                              ...    
ZWE          Zimbabwe        2018                                               30.0    
                             2019                                               30.2    
                             2020                                               30.3    
                             2021                                               30.3    
                             2022                                                NaN    

feature                            access_to_electricity  agricultural_land  \
country_code country_name_wb year                                             
AFG          Afghanistan     1960                    NaN                NaN   
                             1961                    NaN          57.878356   
                             1962                    NaN          57.955016   
                             1963                    NaN          58.031676   
                             1964                    NaN          58.116002   
...                                                  ...                ...   
ZWE          Zimbabwe        2018              45.400288          41.876696   
                             2019              46.682095          41.876696   
                             2020              52.747667          41.876696   
                             2021              48.979927          41.876696   
                             2022                    NaN                NaN   

feature                            agriculture  annual_freshwater_withdrawals  \
country_code country_name_wb year                                               
AFG          Afghanistan     1960          NaN                            NaN   
                             1961          NaN                            NaN   
                             1962          NaN                            NaN   
                             1963          NaN                            NaN   
                             1964          NaN                            NaN   
...                                        ...                            ...   
ZWE          Zimbabwe        2018     7.319375                      30.761677   
                             2019     9.819262                      30.761677   
                             2020     8.772859                      30.761677   
                             2021     8.849899                            NaN   
                             2022          NaN                            NaN   

feature                            annualized_average_growth_rate_in_per_capita_real_survey  \
country_code country_name_wb year                                                             
AFG          Afghanistan     1960                                                NaN          
                             1961                                                NaN          
                             1962                                                NaN          
                             1963                                                NaN          
                             1964                                                NaN          
...                                                                              ...          


In [18]:
wb = pd.DataFrame(wb.to_records())
wb

,country_code,country_name_wb,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_survey,cause_of_death,...,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality
0,AFG,Afghanistan,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1961,NaN,NaN,57.878356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,1962,NaN,NaN,57.955016,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,1963,NaN,NaN,58.031676,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,1964,NaN,NaN,58.116002,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12217,ZWE,Zimbabwe,2018,30.0,45.400288,41.876696,7.319375,30.761677,NaN,NaN,...,NaN,-1.292463,406.23,5.0,27.214585,NaN,6.784,NaN,-1.136798,NaN
12218,ZWE,Zimbabwe,2019,30.2,46.682095,41.876696,9.819262,30.761677,NaN,47.647301,...,NaN,-1.303515,431.62,6.0,27.214747,NaN,7.370,NaN,-1.163669,NaN
12219,ZWE,Zimbabwe,2020,30.3,52.747667,41.876696,8.772859,30.761677,NaN,NaN,...,NaN,-1.329611,480.16,NaN,27.214747,NaN,7.898,NaN,-1.113408,83.3
12220,ZWE,Zimbabwe,2021,30.3,48.979927,41.876696,8.849899,NaN,NaN,NaN,...,NaN,-1.277202,NaN,NaN,27.214747,NaN,8.067,NaN,-1.135830,NaN


In [19]:
wb = wb.astype({'year': 'int64'})
wb['year']

0        1960
1        1961
2        1962
3        1963
4        1964
         ... 
12217    2018
12218    2019
12219    2020
12220    2021
12221    2022
Name: year, Length: 12222, dtype: int64

### 2h.

Create a new dataframe containing just the rows from the World Bank country-timeseries dataset that refer to the whole world. Then remove the "World" rows from the
World Bank country-time-series dataset. Drop the 'country_code' and
'country_name_wb' from the world dataset. Finally, with the exception of the year
column, rename all columns in the world dataset by adding "world_" in front of the
column name

In [20]:
world = wb.query("country_name_wb == 'World'")
world

,country_code,country_name_wb,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_survey,cause_of_death,...,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality
11844,WLD,World,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11845,WLD,World,1961,NaN,NaN,35.879317,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11846,WLD,World,1962,NaN,NaN,35.952470,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11847,WLD,World,1963,NaN,NaN,36.035383,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11848,WLD,World,1964,NaN,NaN,36.117043,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11902,WLD,World,2018,67.696517,89.893733,36.738458,3.951747,9.137864,NaN,NaN,...,2.196050,NaN,2.565871e+06,5.507853,14.337515,NaN,5.699076,NaN,NaN,NaN
11903,WLD,World,2019,68.921601,90.194716,36.762648,4.013242,9.167900,NaN,18.40885,...,2.331934,NaN,2.734962e+06,5.685864,14.599801,NaN,5.543161,NaN,NaN,NaN
11904,WLD,World,2020,70.184805,90.483629,36.730920,4.352489,9.109511,NaN,NaN,...,2.627081,NaN,2.933011e+06,NaN,14.787111,NaN,6.898273,NaN,NaN,NaN
11905,WLD,World,2021,71.331487,91.413941,36.841665,4.304093,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,14.552260,NaN,6.201871,NaN,NaN,NaN


In [21]:
wb = wb.query("country_name_wb != 'World'")
wb

,country_code,country_name_wb,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_survey,cause_of_death,...,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality
0,AFG,Afghanistan,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1961,NaN,NaN,57.878356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,1962,NaN,NaN,57.955016,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,1963,NaN,NaN,58.031676,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,1964,NaN,NaN,58.116002,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12217,ZWE,Zimbabwe,2018,30.0,45.400288,41.876696,7.319375,30.761677,NaN,NaN,...,NaN,-1.292463,406.23,5.0,27.214585,NaN,6.784,NaN,-1.136798,NaN
12218,ZWE,Zimbabwe,2019,30.2,46.682095,41.876696,9.819262,30.761677,NaN,47.647301,...,NaN,-1.303515,431.62,6.0,27.214747,NaN,7.370,NaN,-1.163669,NaN
12219,ZWE,Zimbabwe,2020,30.3,52.747667,41.876696,8.772859,30.761677,NaN,NaN,...,NaN,-1.329611,480.16,NaN,27.214747,NaN,7.898,NaN,-1.113408,83.3
12220,ZWE,Zimbabwe,2021,30.3,48.979927,41.876696,8.849899,NaN,NaN,NaN,...,NaN,-1.277202,NaN,NaN,27.214747,NaN,8.067,NaN,-1.135830,NaN


In [22]:
world = world.drop(['country_code', 'country_name_wb'],
                  axis = 1)
world

,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_survey,cause_of_death,children_in_employment,co2_emissions,...,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality
11844,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11845,1961,NaN,NaN,35.879317,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11846,1962,NaN,NaN,35.952470,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11847,1963,NaN,NaN,36.035383,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11848,1964,NaN,NaN,36.117043,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11902,2018,67.696517,89.893733,36.738458,3.951747,9.137864,NaN,NaN,NaN,4.641294,...,2.196050,NaN,2.565871e+06,5.507853,14.337515,NaN,5.699076,NaN,NaN,NaN
11903,2019,68.921601,90.194716,36.762648,4.013242,9.167900,NaN,18.40885,NaN,4.582036,...,2.331934,NaN,2.734962e+06,5.685864,14.599801,NaN,5.543161,NaN,NaN,NaN
11904,2020,70.184805,90.483629,36.730920,4.352489,9.109511,NaN,NaN,NaN,4.291853,...,2.627081,NaN,2.933011e+06,NaN,14.787111,NaN,6.898273,NaN,NaN,NaN
11905,2021,71.331487,91.413941,36.841665,4.304093,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,14.552260,NaN,6.201871,NaN,NaN,NaN


### 2i.
Next we will merge the World Bank country-time-series dataset with the V-Dem
dataset, matching on the 'country_code' and 'year' columns. First, write a sentence
stating whether you expect this merge to be one-to-one, many-to-one, one-to-many, or
many-to-many, and describe your rationale. Second, merge the two datasets together in
a way that checks whether your expectation is met, and also allows you to see the rows
that failed to match

**I expect this to be many to one. There are more rows in the world bank dataframe than there are in the vdem data frame.** 

In [23]:
wb

,country_code,country_name_wb,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_survey,cause_of_death,...,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality
0,AFG,Afghanistan,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1961,NaN,NaN,57.878356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,1962,NaN,NaN,57.955016,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,1963,NaN,NaN,58.031676,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,1964,NaN,NaN,58.116002,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12217,ZWE,Zimbabwe,2018,30.0,45.400288,41.876696,7.319375,30.761677,NaN,NaN,...,NaN,-1.292463,406.23,5.0,27.214585,NaN,6.784,NaN,-1.136798,NaN
12218,ZWE,Zimbabwe,2019,30.2,46.682095,41.876696,9.819262,30.761677,NaN,47.647301,...,NaN,-1.303515,431.62,6.0,27.214747,NaN,7.370,NaN,-1.163669,NaN
12219,ZWE,Zimbabwe,2020,30.3,52.747667,41.876696,8.772859,30.761677,NaN,NaN,...,NaN,-1.329611,480.16,NaN,27.214747,NaN,7.898,NaN,-1.113408,83.3
12220,ZWE,Zimbabwe,2021,30.3,48.979927,41.876696,8.849899,NaN,NaN,NaN,...,NaN,-1.277202,NaN,NaN,27.214747,NaN,8.067,NaN,-1.135830,NaN


In [24]:
vdem

,country_code,country_name_vdem,year,democracy
5433,AFG,Afghanistan,1960,0.080
5434,AFG,Afghanistan,1961,0.083
5435,AFG,Afghanistan,1962,0.082
5436,AFG,Afghanistan,1963,0.085
5437,AFG,Afghanistan,1964,0.137
...,...,...,...,...
26150,ZZB,Zanzibar,2017,0.267
26151,ZZB,Zanzibar,2018,0.268
26152,ZZB,Zanzibar,2019,0.266
26153,ZZB,Zanzibar,2020,0.258


In [25]:
wb_vdem_merge = pd.merge(wb, vdem, 
                how = 'outer',
                on = ['country_code', 'year'],
                validate = 'many_to_one',
                indicator = 'matched')
wb_vdem_merge

,country_code,country_name_wb,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_survey,cause_of_death,...,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality,country_name_vdem,democracy,matched
0,AFG,Afghanistan,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,0.080,both
1,AFG,Afghanistan,1961,NaN,NaN,57.878356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,0.083,both
2,AFG,Afghanistan,1962,NaN,NaN,57.955016,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,0.082,both
3,AFG,Afghanistan,1963,NaN,NaN,58.031676,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,0.085,both
4,AFG,Afghanistan,1964,NaN,NaN,58.116002,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,0.137,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12549,ZZB,NaN,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zanzibar,0.267,right_only
12550,ZZB,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zanzibar,0.268,right_only
12551,ZZB,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zanzibar,0.266,right_only
12552,ZZB,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zanzibar,0.258,right_only


### 2j.

After this merge, use the .value_counts() method to see the total number of
observations that were found in both datasets, the number found only in the left dataset,
and the number found only in the right dataset. (If you entered the World Bank dataset
into the merge function first, then "left_only" refers to the rows found in the World Bank
but not V-Dem, and "right_only refers to the rows found in V-Dem but not the World
Bank.) There should be more than 9000 rows that matched, but more than 2000 that
failed to match.

In [26]:
wb_vdem_merge['matched'].value_counts()

matched
both          9976
left_only     2183
right_only     395
Name: count, dtype: int64

In [27]:
left_only = (wb_vdem_merge
                 .query("matched == 'left_only'"))

In [28]:
left_only.groupby(['country_code', 'country_name_wb']).agg(min_year = ('year', 'min'), max_year = ('year', 'max'))

,,min_year,max_year
country_code,country_name_wb,,
AFG,Afghanistan,2022,2022
AGO,Angola,2022,2022
ALB,Albania,2022,2022
AND,Andorra,1960,2022
ARE,United Arab Emirates,1960,2022
...,...,...,...
WSM,Samoa,1960,2022
YEM,"Yemen, Rep.",2022,2022
ZAF,South Africa,2022,2022


In [29]:
right_only = (wb_vdem_merge
                 .query("matched == 'right_only'"))
right_only.groupby(['country_code', 'country_name_vdem']).agg(min_year = ('year', 'min'), max_year = ('year', 'max'))

,,min_year,max_year
country_code,country_name_vdem,,
DDR,German Democratic Republic,1960,1990
HKG,Hong Kong,1960,2021
PSE,Palestine/West Bank,1967,2021
PSG,Palestine/Gaza,1960,2021
SML,Somaliland,1991,2021
TWN,Taiwan,1960,2021
VDR,Republic of Vietnam,1960,1975
XKX,Kosovo,1999,2021
YMD,South Yemen,1960,1990


# I need to include a description of why these data do not matter and are due to data coverage 

### 2k.

### 2l.

In [30]:
timeseries = pd.merge(wb, vdem, 
                how = 'inner',
                on = ['country_code', 'year'])
timeseries

,country_code,country_name_wb,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_survey,cause_of_death,...,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality,country_name_vdem,democracy
0,AFG,Afghanistan,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,0.080
1,AFG,Afghanistan,1961,NaN,NaN,57.878356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,0.083
2,AFG,Afghanistan,1962,NaN,NaN,57.955016,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,0.082
3,AFG,Afghanistan,1963,NaN,NaN,58.031676,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,0.085
4,AFG,Afghanistan,1964,NaN,NaN,58.116002,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,0.137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9971,ZWE,Zimbabwe,2017,29.8,43.979065,41.876696,8.340969,27.234910,-3.5,NaN,...,334.71,5.0,27.214585,NaN,6.279,NaN,-1.195905,76.5,Zimbabwe,0.295
9972,ZWE,Zimbabwe,2018,30.0,45.400288,41.876696,7.319375,30.761677,NaN,NaN,...,406.23,5.0,27.214585,NaN,6.784,NaN,-1.136798,NaN,Zimbabwe,0.305
9973,ZWE,Zimbabwe,2019,30.2,46.682095,41.876696,9.819262,30.761677,NaN,47.647301,...,431.62,6.0,27.214747,NaN,7.370,NaN,-1.163669,NaN,Zimbabwe,0.293
9974,ZWE,Zimbabwe,2020,30.3,52.747667,41.876696,8.772859,30.761677,NaN,NaN,...,480.16,NaN,27.214747,NaN,7.898,NaN,-1.113408,83.3,Zimbabwe,0.293


## Question 3

In [31]:
# Go through each of the tables and identify if they are in thrid normal form

### Country

In [32]:
country

,country_code,country_name_wb,country_longname,currency_unit,region,income_group
0,AFG,Afghanistan,Islamic State of Afghanistan,Afghan afghani,South Asia,Low income
1,AGO,Angola,People's Republic of Angola,Angolan kwanza,Sub-Saharan Africa,Lower middle income
2,ALB,Albania,Republic of Albania,Albanian lek,Europe & Central Asia,Upper middle income
3,AND,Andorra,Principality of Andorra,Euro,Europe & Central Asia,High income
5,ARE,United Arab Emirates,United Arab Emirates,U.A.E. dirham,Middle East & North Africa,High income
...,...,...,...,...,...,...
234,WSM,Samoa,Samoa,Samoan tala,East Asia & Pacific,Lower middle income
235,YEM,"Yemen, Rep.",Republic of Yemen,Yemeni rial,Middle East & North Africa,Low income
236,ZAF,South Africa,Republic of South Africa,South African rand,Sub-Saharan Africa,Upper middle income
237,ZMB,Zambia,Republic of Zambia,New Zambian kwacha,Sub-Saharan Africa,Lower middle income


First Normal Form
* Primary key
    * The primary key is the country code
* Everything is atomic
* No repeating groups

Second Normal Form
* No functional dependence

Third Normal Form
* No transitive dependence

### world

In [33]:
world.columns

Index(['year', 'access_to_clean_fuels_and_technologies_for_cooking',
       'access_to_electricity', 'agricultural_land', 'agriculture',
       'annual_freshwater_withdrawals',
       'annualized_average_growth_rate_in_per_capita_real_survey',
       'cause_of_death', 'children_in_employment', 'co2_emissions',
       'coastal_protection', 'control_of_corruption', 'cooling_degree_days',
       'economic_and_social_rights_performance_score',
       'electricity_production_from_coal_sources', 'energy_imports',
       'energy_intensity_level_of_primary_energy', 'energy_use',
       'fertility_rate', 'food_production_index', 'forest_area',
       'fossil_fuel_energy_consumption', 'freshwater_withdrawal', 'gdp_growth',
       'ghg_net_emissions', 'gini_index', 'government_effectiveness',
       'government_expenditure_on_education', 'gross_school_enrollment',
       'heat_index_35', 'heating_degree_days', 'hospital_beds',
       'income_share_held_by_lowest_20pct', 'individuals_using_the_int

First Normal Form
* Primary key
    * The primary key is year
* Everything is atomic
* No repeating groups

Second Normal Form
* No functional dependence

Third Normal Form
* No transitive dependence

### timeseries

In [34]:
timeseries.columns

Index(['country_code', 'country_name_wb', 'year',
       'access_to_clean_fuels_and_technologies_for_cooking',
       'access_to_electricity', 'agricultural_land', 'agriculture',
       'annual_freshwater_withdrawals',
       'annualized_average_growth_rate_in_per_capita_real_survey',
       'cause_of_death', 'children_in_employment', 'co2_emissions',
       'coastal_protection', 'control_of_corruption', 'cooling_degree_days',
       'economic_and_social_rights_performance_score',
       'electricity_production_from_coal_sources', 'energy_imports',
       'energy_intensity_level_of_primary_energy', 'energy_use',
       'fertility_rate', 'food_production_index', 'forest_area',
       'fossil_fuel_energy_consumption', 'freshwater_withdrawal', 'gdp_growth',
       'ghg_net_emissions', 'gini_index', 'government_effectiveness',
       'government_expenditure_on_education', 'gross_school_enrollment',
       'heat_index_35', 'heating_degree_days', 'hospital_beds',
       'income_share_held_by

First Normal Form
* Primary key
    * The primary key is country_code, year
* Everything is atomic
* No repeating groups

Second Normal Form
* No functional dependence

Third Normal Form
* No transitive dependence

Don't need to include country_name_vdem since we have country_name_wb in the country table. Can drop both from the timeseries table.

In [35]:
timeseries = timeseries.drop(['country_name_wb', 'country_name_vdem'], axis = 1)

In [36]:
timeseries.columns

Index(['country_code', 'year',
       'access_to_clean_fuels_and_technologies_for_cooking',
       'access_to_electricity', 'agricultural_land', 'agriculture',
       'annual_freshwater_withdrawals',
       'annualized_average_growth_rate_in_per_capita_real_survey',
       'cause_of_death', 'children_in_employment', 'co2_emissions',
       'coastal_protection', 'control_of_corruption', 'cooling_degree_days',
       'economic_and_social_rights_performance_score',
       'electricity_production_from_coal_sources', 'energy_imports',
       'energy_intensity_level_of_primary_energy', 'energy_use',
       'fertility_rate', 'food_production_index', 'forest_area',
       'fossil_fuel_energy_consumption', 'freshwater_withdrawal', 'gdp_growth',
       'ghg_net_emissions', 'gini_index', 'government_effectiveness',
       'government_expenditure_on_education', 'gross_school_enrollment',
       'heat_index_35', 'heating_degree_days', 'hospital_beds',
       'income_share_held_by_lowest_20pct', 'in

# 4

In [37]:
dbserver = psycopg2.connect(
    host = 'postgres',
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    port = 5432
)
dbserver.autocommit = True

In [38]:
cursor = dbserver.cursor()

In [39]:
try:
    cursor.execute("CREATE DATABASE cardib")
except:
    cursor.execute("DROP DATABASE cardib")
    cursor.execute("CREATE DATABASE cardib")

In [40]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    host = 'postgres',
    port = 5432,
    db = 'cardib'
))

In [41]:
print(country.shape[0])
country.to_sql('country', 
               con = engine, 
               index = False, 
               chunksize = 1000, 
               if_exists = 'replace')

193


193

In [42]:
print(world.shape[0])
world.to_sql('world', 
               con = engine, 
               index = False, 
               chunksize = 1000, 
               if_exists = 'replace')

63


63

In [43]:
print(timeseries.shape[0])
timeseries.to_sql('timeseries', 
               con = engine, 
               index = False, 
               chunksize = 100, 
               if_exists = 'replace')

9976


9976

# 5) Dbdocs ER Diagram

# 6) Querries

a) What countries had the highest quality democracies in the year 2021? Join the
timeseries and country tables, select the country_name_wb and democracy attributes,
rename county_name_wb to country, filter the rows to only those for the year 2021, and
sort the rows in descending order by the value of democracy

In [49]:
myquery = '''
SELECT country_name_wb as country,
        democracy
FROM timeseries t
INNER JOIN country c
ON t.country_code = c.country_code
WHERE t.year = 2021
ORDER BY t.democracy
    desc
'''
pd.read_sql_query(myquery, con = engine)

,country,democracy
0,Denmark,0.915
1,Sweden,0.903
2,Norway,0.901
3,Costa Rica,0.898
4,Switzerland,0.898
...,...,...
167,Qatar,0.090
168,"Korea, Dem. People's Rep.",0.086
169,China,0.077
170,Eritrea,0.072


b. How does the life expectancy at birth for Chile compare to the global average life
expectancy at birth over the 1960-2021 time span? Join the timeseries and world tables,
select year from the timeseries table and the life expectancy attributes from both the
timeseries and world tables, filter the rows to just those where the country code is CHL,
and sort the rows by year.

In [56]:
myquery = '''
SELECT t.year, 
        t.life_expectancy_at_birth as chl_life_expectancy,
        w.life_expectancy_at_birth as world_life_expectancy
FROM timeseries t
INNER JOIN world w
    ON t.year = w.year
WHERE t.country_code = 'CHL'
ORDER BY t.year
    ASC
'''
pd.read_sql_query(myquery, con = engine)

,year,chl_life_expectancy,world_life_expectancy
0,1960,57.015,50.894180
1,1961,57.537,52.846336
2,1962,57.771,55.208684
3,1963,57.150,55.542341
4,1964,58.738,56.034875
...,...,...,...
57,2017,80.350,72.542776
58,2018,80.133,72.784090
59,2019,80.326,72.979716
60,2020,79.377,72.243822


c. What regions of the world generated the most carbon dioxide emissions in 2019? Join
the timeseries and country tables, filter the rows to only the ones that refer to 2019, and
aggregate the data by region. Keep the region column from the country table and the
sum of the co2_emissions attribute from the timeseries table. Rename the sum to
co2_emissions. Then sort the rows in descending order by co2_emissions.

In [60]:
myquery = '''
SELECT c.region, 
        SUM(t.co2_emissions) as co2_emissions
FROM timeseries t
INNER JOIN country c
    ON t.country_code = c.country_code
WHERE t.year = 2019
GROUP BY c.region
ORDER BY co2_emissions
    DESC
'''
pd.read_sql_query(myquery, con = engine)

,region,co2_emissions
0,Europe & Central Asia,273.807274
1,Middle East & North Africa,176.008097
2,East Asia & Pacific,94.256987
3,Latin America & Caribbean,68.692584
4,Sub-Saharan Africa,43.318399
5,North America,29.726128
6,South Asia,10.820011


d. What countries experienced the greatest increases in democratic quality over the
1960-2021 time span? Use subqueries to create two new tables on the fly. One table
contains country_code and democracy from the timeseries table in which only the rows
from 1960 are present. The second table contains country_code and democracy from
the timeseries table in which only the rows from 2021 are present. Join these two tables
together, then join the result with the country table. Select the country name from
county, democracy from the 1960 table, democracy from the 2021 table, and the
difference between the two democracy scores. Give each column a more intuitive name.
Then sort the rows in descending order by the democracy difference and display the
first 10 rows.

e. By count of countries, what is the most commonly used currency in the world?

f. The GINI index measures the amount of economic inequality in a country. The higher
the index, the greater the economic disparity between rich and poor. How does the
average GINI index compare across income groups (see the country table) in 2019?